# README
How to install Jupyter: https://jupyter.org/install \
Where to download the data: https://github.com/patiegm/Udacity_Data_Analysis_Nanodegree/tree/master/Analyze%20AB%20Test%20Results
# Context
Let's assume we run an experiment by randomly assigning users to treatment vs. control. While the users in treatment were exposed to the new page, the users in control still saw the old page. We want to understand whether the new page significantly improved user conversion defined by the percentage of users who converted after being exposed to the new vs. old page. Besides, we also have indivudal users' countries to perform variance reduction.

In [1]:
import pandas as pd
from scipy import stats
from scipy.stats import t
from scipy.stats import norm
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# Load Data

In [2]:
data = pd.read_csv('ab_data.csv')

In [3]:
country = pd.read_csv('countries.csv')

In [4]:
data = pd.merge(data, country, on='user_id', how='inner')

In [5]:
data

,user_id,timestamp,group,landing_page,converted,country
0,851104,2017-01-21 22:11:48.556739,control,old_page,0,US
1,804228,2017-01-12 08:01:45.159739,control,old_page,0,US
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0,US
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0,US
4,864975,2017-01-21 01:52:26.210827,control,old_page,1,US
...,...,...,...,...,...,...
294473,751197,2017-01-03 22:28:38.630509,control,old_page,0,US
294474,945152,2017-01-12 00:51:57.078372,control,old_page,0,US
294475,734608,2017-01-22 11:45:03.439544,control,old_page,0,US
294476,697314,2017-01-15 01:20:28.957438,control,old_page,0,US


# Clean Data

In [6]:
def remove_mixed_assignment(df):
    df1 = df[['user_id', 'group']].groupby(['user_id']).nunique().reset_index() 
    # count the unique number of groups that a user was assigned to
    df2 = pd.merge(df, df1, on=['user_id'], how='left') 
    return df2[df2['group_y'] == 1][['user_id', 'country', 'timestamp', 'group_x', 'landing_page', 'converted']]\
        .rename(columns={'group_x':'group'})
    # only return users assigned to either treatment or control

In [7]:
data1 = remove_mixed_assignment(data)

In [8]:
def remove_exposure_bugs(df):
    df1 = df[(df['group'] == 'control')&(df['landing_page'] == 'new_page')][['user_id', 'group']] 
    # identify the users in control expposed to treatment
    df2 = df[(df['group'] == 'treatment')&(df['landing_page'] == 'old_page')][['user_id', 'group']] 
    # identify the users in treatment expposed to control
    df3 = pd.concat([df1, df2])
    df4 = pd.merge(df, df3, on=['user_id'], how='left')
    return df4[df4['group_y'].isna()][['user_id', 'country', 'timestamp', 'group_x', 'landing_page', 'converted']]\
        .rename(columns={'group_x':'group'})
    # only return users with the correct exposure

In [9]:
data2 = remove_exposure_bugs(data1)

In [10]:
def consolidate_multiple_exposures(df):
    df1 = df.groupby(['user_id', 'country', 'group', 'landing_page'])\
        .agg({'timestamp': ['min', 'max'], 'converted': ['count', 'sum']}) 
    # get the timestamps of the first and last exposure, the number of exposures and the number of conversions
    df1.columns = df1.columns.droplevel(0)
    df2 = df1.reset_index()
    df2['converted'] = df2.apply(lambda x: int(x['sum'] > 0), axis=1) # 1 if the user has one conversion
    df2['conversion_rate'] = 1.0*df2['sum']/df2['count'] # the number of conversions divided by the number of exposures
    return df2
    # one user will only have one row

In [11]:
data3 = consolidate_multiple_exposures(data2)

In [12]:
data3

,user_id,country,group,landing_page,min,max,count,sum,converted,conversion_rate
0,630000,US,treatment,new_page,2017-01-19 06:26:06.548941,2017-01-19 06:26:06.548941,1,0,0,0.0
1,630001,US,treatment,new_page,2017-01-16 03:16:42.560309,2017-01-16 03:16:42.560309,1,1,1,1.0
2,630002,US,control,old_page,2017-01-19 19:20:56.438330,2017-01-19 19:20:56.438330,1,0,0,0.0
3,630003,US,treatment,new_page,2017-01-12 10:09:31.510471,2017-01-12 10:09:31.510471,1,0,0,0.0
4,630004,US,treatment,new_page,2017-01-18 20:23:58.824994,2017-01-18 20:23:58.824994,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...
286686,945994,UK,control,old_page,2017-01-03 14:41:21.565258,2017-01-03 14:41:21.565258,1,0,0,0.0
286687,945996,US,treatment,new_page,2017-01-09 18:58:19.952277,2017-01-09 18:58:19.952277,1,0,0,0.0
286688,945997,US,control,old_page,2017-01-04 06:56:24.658147,2017-01-04 06:56:24.658147,1,0,0,0.0
286689,945998,CA,control,old_page,2017-01-16 07:08:02.207969,2017-01-16 07:08:02.207969,1,0,0,0.0


# Z Test

In [87]:
def perform_z_test(df):
    n_treatment = df[df['group'] == 'treatment']['user_id'].count() # the number of users in treatment
    n_control = df[df['group'] == 'control']['user_id'].count() # the number of users in control
    
    p_treatment = 1.0*df[df['group'] == 'treatment']['converted'].sum()/n_treatment 
    # the probability of a user in treatment to convert
    p_control = 1.0*df[df['group'] == 'control']['converted'].sum()/n_control 
    # the probability of a user in control to convert
    
    p_delta = p_treatment - p_control # the delta of the probability of a user to convert in treatment vs. control
    print('The treatment effect is', p_delta)
    
    p = 1.0*df['converted'].sum()/(n_treatment+n_control)
    # the probability of a user to convert under null hypothesis
    var = p*(1-p)
    # the variance of the probability of a user to convert under null hypothesis
    se = np.sqrt(var*(1/n_treatment + 1/n_control))
    # the pooled standard error of the z test
    print('The standard error is', se)
    
    z_statistic = p_delta/se # the z statistic
    
    pvalue = 2*norm.cdf(-abs(z_statistic)) # the p value of the z test
    print('The p value is', pvalue)
    
    lower = p_delta - norm.ppf(0.975)*se # the lower bound of the confidence interval
    upper = p_delta + norm.ppf(0.975)*se # the upper bound of the confidence interval
    print('The lower bound is', lower)
    print('The upper bound is', upper)

In [88]:
perform_z_test(data3)

The treatment effect is -0.0014478458686042056
The standard error is 0.0012114117107354344
The p value is 0.232019670931933
The lower bound is -0.003822169192095711
The upper bound is 0.0009264774548872999


# Stratified Sampling
We use countries to divide users into strata and calculate the treatment effect and variance within each strata. \
Then we can use weighted sum to calculate the overall treatment effect and standard error between treatment vs. control. \
Here by assuming there is no between-strata variance, we effectively reduce the standard error. 

In [84]:
def performa_stratified_sampling(df):
    n_treatment_us, n_control_us, p_delta_us, var_us = calculate_each_strata(df, 'US')
    n_treatment_uk, n_control_uk, p_delta_uk, var_uk = calculate_each_strata(df, 'UK')
    n_treatment_ca, n_control_ca, p_delta_ca, var_ca = calculate_each_strata(df, 'CA')
    # calculate the sample size, delta and variance for each strata
    
    n_treatment = df[df['group'] == 'treatment']['user_id'].count() # the number of users in treatment
    n_control = df[df['group'] == 'control']['user_id'].count() # the number of users in control
    
    w_us = (n_treatment_us + n_control_us)/(n_treatment+n_control)
    w_uk = (n_treatment_uk + n_control_uk)/(n_treatment+n_control)
    w_ca = (n_treatment_ca + n_control_ca)/(n_treatment+n_control)
    # calculate the weight for each strata
    
    p_delta = p_delta_us*w_us + p_delta_uk*w_uk + p_delta_ca*w_ca # calculated the weighted delta
    print('The treatment effect is', p_delta)
    
    se = np.sqrt((var_us*w_us + var_uk*w_uk + var_ca*w_ca)*(1/n_treatment + 1/n_control)) 
    # calculated the weighted standard error
    print('The standard error is', se)
    
    z_statistic = p_delta/se # the z statistic
    
    pvalue = 2*norm.cdf(-abs(z_statistic)) # the p value of the z test
    print('The p value is', pvalue)
    
    lower = p_delta - norm.ppf(0.975)*se # the lower bound of the confidence interval
    upper = p_delta + norm.ppf(0.975)*se # the upper bound of the confidence interval
    print('The lower bound is', lower)
    print('The upper bound is', upper)

In [85]:
def calculate_each_strata(df, country):
    n_treatment = df[(df['group'] == 'treatment')&(df['country'] == country)]\
    ['user_id'].count() # the number of users in treatment in the strata
    n_control = df[(df['group'] == 'control')&(df['country'] == country)]\
    ['user_id'].count() # the number of users in control in the strata
    pvalue = stats.binom_test(n_treatment, 
                              n_treatment + n_control, 
                              p=0.5, 
                              alternative='two-sided') 
    # test whether the treatment vs. control in the strata has the equal sample size
    print(pvalue)
    
    p_treatment = 1.0*df[(df['group'] == 'treatment')&(df['country'] == country)]\
    ['converted'].sum()/n_treatment 
    # the probability of a user in treatment to convert in the strata
    p_control = 1.0*df[(df['group'] == 'control')&(df['country'] == country)]\
    ['converted'].sum()/n_control 
    # the probability of a user in control to convert in the strata
    
    p_delta = p_treatment - p_control 
    # the delta of the probability of a user to convert in treatment vs. control in the strata
    
    p = 1.0*df[df['country'] == country]['converted'].sum()/(n_treatment+n_control)
    # the probability of a user to convert under null hypothesis in the strata
    var = p*(1-p)
    # the variance of the probability of a user to convert under null hypothesis in the strata
    
    return n_treatment, n_control, p_delta, var

In [86]:
performa_stratified_sampling(data3)

0.5892325882523092
0.3460181962779027
0.3404809447553442
The treatment effect is -0.0014429556962448117
The standard error is 0.001211406148937557
The p value is 0.23359818335457794
The lower bound is -0.003817268118812788
The upper bound is 0.000931356726323165


# CUPED
We use whether the user is in CA as the control variate to perform CUPED. \
Here $y = 1$ if converted and 0 if not and $x = 1$ if in CA and 0 if not. \
Using CUPED, we have $\hat{y} = y - \theta x + \theta \bar{x}$ to compare the treatment vs. contorl.

In [124]:
def perform_cuped(df):
    df['variate'] = df.apply(lambda x: int(x['country'] == 'CA'), axis=1)
    # create the control variate based on whether the use is in CA
    covariance = np.cov(df['converted'], df['variate']) 
    # calculate the covariance matrix between the metric (converted) and the variate
    theta = covariance[0][1]/covariance[1][1] # calculate the theta
    df['converted_adjusted'] = df['converted'] - theta*df['variate'] + theta*df['variate'].mean()
    # adjust the metric (converted) based on the control variate
    
    n_treatment = df[df['group'] == 'treatment']['user_id'].count() # the number of users in treatment
    n_control = df[df['group'] == 'control']['user_id'].count() # the number of users in control
    
    mu_treatment = 1.0*df[df['group'] == 'treatment']['converted_adjusted'].mean()
    # the mean of the adjusted metric among users in treatment
    mu_control = 1.0*df[df['group'] == 'control']['converted_adjusted'].mean()
    # the mean of the adjusted metric among users in control
    
    mu_delta = mu_treatment - mu_control # the delta of the adjusted metric mean among users in treatment vs. control
    print('The treatment effect is', mu_delta)
    
    var = df['converted_adjusted'].var() # the variance of the adjusted metric under null hypothesis
    se = np.sqrt(var*(1/n_treatment + 1/n_control))
    # the pooled standard error of the z test
    print('The standard error is', se)
    
    z_statistic = mu_delta/se # the z statistic
    
    pvalue = 2*norm.cdf(-abs(z_statistic)) # the p value of the z test
    print('The p value is', pvalue)
    
    lower = mu_delta - norm.ppf(0.975)*se # the lower bound of the confidence interval
    upper = mu_delta + norm.ppf(0.975)*se # the upper bound of the confidence interval
    print('The lower bound is', lower)
    print('The upper bound is', upper)

In [125]:
perform_cuped(data3)

The treatment effect is -0.0014447760459490033
The standard error is 0.0012114094360929464
The p value is 0.23301016807988406
The lower bound is -0.0038190949112231545
The upper bound is 0.0009295428193251479
